# Secure Envelopes Viewer

This notebook is designed to assist debugging an envoy node in order to review the data in a secure envelope.

To use this notebook, make sure you have the following environment variables set:

- `$ENVOY_URL`
- `$ENVOY_CLIENT_ID`
- `$ENVOY_CLIENT_SECRET`

You can also store these variables in a `.env` file in the current working directory where you're running jupyter.

## Dependencies

You'll need to install the following dependencies for this notebook to work:

```
$ pip install pyenvoy tabulate
```

In [6]:
import json

from envoy import connect
from tabulate import tabulate

In [7]:
envoy = connect()
envoy.status()

{'status': 'ok',
 'uptime': '218h6m29.086308804s',
 'version': '0.24.0-beta.28 (019fd7e)'}

## List Transactions

All secure envelopes are part of transactions, first find the ID of the transaction you're looking for.

In [13]:
def list_transactions():
    txns = envoy.transactions.list()
    
    table = [["ID", "Status", "Counterparty", "Network", "Amount", "Envelopes"]]
    fields = ["id", "status", "counterparty", "virtual_asset", "amount", "envelope_count"]
    
    for txn in txns:
        table.append([txn[field] for field in fields])
    
    print(tabulate(table, headers='firstrow'))


list_transactions()

ID                                    Status    Counterparty        Network          Amount    Envelopes
------------------------------------  --------  ------------------  ---------  ------------  -----------
34cc8173-0f6f-45d7-815f-6682afb9dd6c  review    UAB Raison Markets  Ethereum    0.01                   2
e4d8311b-690e-42a5-bdd4-0e07bb989521  pending   Yellow Card, Inc.   BTC         3.54e-05               2
1617520d-8d27-422e-ba59-b36a5701ede6  pending   CharlieVASP         BTC         3.54e-05               2
82d3aa97-44a3-4445-8b7d-da87bc93a699  rejected  CharlieVASP         BTC         0.000612664            4
7effc231-3045-42f5-a073-90c650c3fc49  review    CharlieVASP         ETH (GAS)  23                      4
36af060d-147b-477f-99f5-ed739c3b5260  review    CharlieVASP         BTC         1e-06                  4


## List Secure Envelopes

List the secure envelopes for a specific transaction in order to get the secure envelope ID.

In [19]:
def list_secure_envelopes(txn_id):
    txn = envoy.transactions.detail(txn_id)
    envs = txn.secure_envelopes.list()

    table = [["ID", "Direction", "Reply To", "Is Error", "Timestamp", "Transfer State"]]
    fields = ["id", "direction", "reply_to", "is_error", "timestamp", "transfer_state"]

    for env in envs:
        row = [env[field] for field in fields]
        if row[2] == "00000000000000000000000000":
            row[2] = ""
        
        table.append(row)
    
    print(tabulate(table, headers='firstrow'))


list_secure_envelopes("1617520d-8d27-422e-ba59-b36a5701ede6")

ID                          Direction    Reply To                    Is Error    Timestamp                       Transfer State
--------------------------  -----------  --------------------------  ----------  ------------------------------  ----------------
01J3ZJ2MMHDRJE13DW284M5727  in           01J3ZJ2MMHDRJE13DW25B8RDY4  False       2024-07-29T15:34:52.303915438Z  PENDING
01J3ZJ2MMHDRJE13DW25B8RDY4  out                                      False       2024-07-29T15:34:51.985011836Z  STARTED


## Show Secure Envelope Payload

Specify the transaction ID and secure envelope ID to view the decrypted data in the secure envelope.

In [20]:
def decrypt_secure_envelope(txn_id, env_id):
    txn = envoy.transactions.detail(txn_id)
    env = txn.secure_envelopes.detail(env_id, params={'decrypt': True})

    print(json.dumps(env.data, indent=2))


decrypt_secure_envelope("1617520d-8d27-422e-ba59-b36a5701ede6", "01J3ZJ2MMHDRJE13DW25B8RDY4")

{
  "id": "01J3ZJ2MMHDRJE13DW25B8RDY4",
  "envelope_id": "1617520d-8d27-422e-ba59-b36a5701ede6",
  "direction": "out",
  "remote": "api.charlie.vaspbot.net",
  "reply_to": "00000000000000000000000000",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tilcott",
                  "secondaryIdentifier": "Mary",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ]
              }
            ],
            "nationalIdentification": {
              "countryOfIssue": "GB"
            },
            "dateAndPlaceOfBirth": {
              "dateOfBirth": "1972-